In [21]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging

logger = logging.getLogger()
handler = logger.handlers[0]
handler.setLevel(logging.CRITICAL)

---

## User Creation and Login

In [22]:
%run ../scripts/create_user.py --test

Enter user name: Burton DeWilde
Enter user email: burtdewilde@gmail.com
Confirm user email: burtdewilde@gmail.com
Enter password: ········
Confirm password: ········


2016-07-08 14:10:44,854 - create_user - INFO - created user (TEST): {'email': 'burtdewilde@gmail.com', 'password': 'test', 'name': 'Burton DeWilde', 'owned_project_ids': None, 'project_ids': None, 'created_ts': '2016-07-07T18:50:49.381497Z'}


In [26]:
%run ../scripts/login_user.py

Enter email: burtdewilde@gmail.com
Enter password: ········


2016-07-08 14:13:41,337 - login_user - INFO - Welcome, Burton DeWilde! id=1


---

## Project Creation and Joining

In [24]:
%run ../scripts/create_project.py --user_id=1 --test

Enter project name: foo
Enter project description (optional): bar


2016-07-08 14:11:33,108 - create_project - INFO - created project (TEST): {'created_ts': '2016-07-07T18:50:49.380396Z', 'name': 'foo', 'owner_user_id': 1, 'description': 'bar', 'user_ids': [1]}


In [25]:
%run ../scripts/delete_project.py --user_id=1 --project_id=4 --test

2016-07-08 14:11:52,152 - delete_project - INFO - deleted project id=4 (TEST)
